1) 웹드라이버 객체 할당

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd

driver = webdriver.Chrome("./chromedriver")
driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="f3395dce04d463771e9938bfd7ab9307")>

2) 아마존 웹사이트 열기

In [34]:
main_url = "https://www.amazon.com/"
driver.get(main_url)

3) 검색 키워드 설정 및 검색

In [35]:
keyword = "imac 24inch 2021"

sel_search = driver.find_element_by_id("twotabsearchtextbox")
sel_btn = driver.find_element_by_id("nav-search-submit-button")

# print(sel_search.text, sel_btn.text)
sel_search.clear()
sel_search.send_keys(keyword)
sel_btn.click()

4) 목록 중 원하는 상품 선택 및 모든 리뷰 보기 페이지로 이동

In [36]:
page = driver.page_source
soup = BeautifulSoup(page, 'html.parser')
all_soup = soup.find_all("div",class_="s-main-slot s-result-list s-search-results sg-row")[0]
one_soup = all_soup.find_all("div", class_="s-result-item s-asin sg-col-0-of-12 sg-col-16-of-20 sg-col sg-col-12-of-16")[0]
href_soup = one_soup.find("a", class_="a-link-normal a-text-normal")
prod_url = main_url +href_soup.get("href")
# print(prod_url)

In [78]:
driver.get(prod_url)
sel_r = driver.find_element_by_xpath('//*[@id="acrCustomerReviewText"]')
sel_r.click()

sel_r_all = driver.find_element_by_xpath('//*[@id="reviews-medley-footer"]/div[2]/a')
sel_r_all.click()

5) 원하는 정보 가져오기

In [94]:
# 한 페이지, 한 개의 리뷰만
page = driver.page_source
soup = BeautifulSoup(page, 'html.parser')
all_soup = soup.find_all("div", class_="a-section review aok-relative")

one_soup = all_soup[1]

usr = one_soup.find("span", class_="a-profile-name").text
score = one_soup.find("span", class_="a-icon-alt").text[:3]
title = one_soup.find("a", class_="a-size-base a-link-normal review-title a-color-base review-title-content a-text-bold").span.text
info = one_soup.find("span", class_="a-size-base a-color-secondary review-date").text
body = one_soup.find("span", class_="a-size-base review-text review-text-content").span.text.strip()

date = info.split(" ")[-3:]
date = " ".join(date)

cntry = info.split(" ")[3:-4]
cntry = " ".join(cntry)

npp = one_soup.find("span", class_="a-size-base a-color-tertiary cr-vote-text").text
npp = npp.split(" ")[0]

print(usr, score, date, cntry, npp, title, body)

In [ ]:
# 여러 페이지, 여러 리뷰
url_1page = 'https://www.amazon.com/Apple-24-inch-8%E2%80%91core-7%E2%80%91core-256GB/product-reviews/B0932FPBV8/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews'
driver.get(url_1page)

usr_all = []
score_all = []
title_all = []
date_all = []
cntry_all = []
npp_all = []
body_all = []

while True:
    
    time.sleep(3)
    page = driver.page_source
    soup = BeautifulSoup(page, "html.parser")
    all_soup = soup.find_all("div", class_="a-section review aok-relative")
    
    
    for one in all_soup :
        usr = one.find("span", class_="a-profile-name").text
        score = one.find("span", class_="a-icon-alt").text[:3]
        title = one.find("a", class_="a-size-base a-link-normal review-title a-color-base review-title-content a-text-bold").span.text
        info = one.find("span", class_="a-size-base a-color-secondary review-date").text
        body = one.find("span", class_="a-size-base review-text review-text-content").span.text.strip()
        
        date = info.split(" ")[-3:]
        date = " ".join(date)

        cntry = info.split(" ")[3:-4]
        cntry = " ".join(cntry)
        try :
            npp = one.find("span", class_="a-size-base a-color-tertiary cr-vote-text").text
            npp = npp.split(" ")[0]
            npp_all.append(npp)
        except :
            npp = "0"
            npp_all.append(npp)

        usr_all.append(usr)
        score_all.append(score)
        title_all.append(title)
        body_all.append(body)
        date_all.append(date)
        cntry_all.append(cntry)
    
    try :
        sel_next = driver.find_element_by_xpath('//*[@id="cm_cr-pagination_bar"]/ul/li[2]/a')
        sel_next.click()
    except :
        break

6) DataFrame으로 만들어 csv로 저장하기

In [106]:
dat = {"usr_id":usr_all, "score":score_all,"date":date_all,"country":cntry_all,"review_title": title_all,  "review_body": body_all} 
dat = pd.DataFrame(dat)
dat.to_csv("imac_review.csv")
dat

,usr_id,score,date,country,review_title,review_body
0,daviddiy,5.0,"June 2, 2021",United States,Best iMac so far,I’m downsizing from a 27in iMac to this 24in a...
1,Nick,1.0,"May 28, 2021",United States,Strong mold smell,Computer had a very strong mold smell due to s...
2,Linda K.,1.0,"June 6, 2021",United States,Buy your computers from Amazon,3 weeks after buying this computer I was asked...
3,Rolando Pizarro,5.0,"May 30, 2021",United States,Best iMac ever made!,I ordered the iMac since the spring loaded eve...
4,Larry,5.0,"June 7, 2021",United States,It works flawlessly,I have favored using Apple products over the y...
5,Michael U.,4.0,"June 11, 2021",United States,Love the iMac but have already had problems wi...,I have used the iMac for years and enjoy worki...
6,Oregon T.,1.0,"June 11, 2021",United States,Worked fine for a couple days then BRICK!,Updated to newest OS - Worked for a couple day...
7,Amazon Customer,5.0,"June 13, 2021",United States,Greta. Clean and Sleek,"If you can afford to, get the one with the mos..."
8,CAtoAKgal,5.0,"June 14, 2021",United States,The feel of ease,Both Apple amd Amazon strive to make it easy. ...
9,Ray,5.0,"May 27, 2021",United States,Refreshing green. Buy the base model!,..Unless you are a “pro” user utilizing high l...
